In [1]:
import plotly.express as px
import pandas as pd
from datetime import datetime, timedelta

In [2]:
overall_rounds = 52
rounds_fuel = 14
rounds_tires = 13
tires_perc_problem = .3
round_duration_sec = 51
secs_lost_with_tyres_low_per_round = 5

__fuel_amount_start__ = 100
__tire_change_sec__ = 3
__l_per_sec__ = 5

strats = ['avoid_low_tyre', 'minimize_low_tyre', 'lower_stops_ap']

In [8]:
rounds_without_tire_problem = rounds_tires * (1- tires_perc_problem)
stints_needed = overall_rounds /  rounds_without_tire_problem

print(f"Rounds per stint: {rounds_without_tire_problem} - in {stints_needed}")

Rounds per stint: 9.1 - in 5.714285714285714


In [10]:
splits = []
rounds_driven = 0
i = 1
start_fuel = 100
now = datetime.utcnow()

while True:    
    ############
    # ROUND CALC #
    ############
    if i == 1:
        start = f"{now.isoformat()[:-3]}Z"
        end = f"{(now +  timedelta(seconds=rounds_without_tire_problem * round_duration_sec)).isoformat()[:-3]}Z"
    else:
        start =  f"{datetime.strptime(end, '%Y-%m-%dT%H:%M:%S.%fZ').isoformat()[:-3]}Z"
        end = f"{(datetime.strptime(end, '%Y-%m-%dT%H:%M:%S.%fZ') +  timedelta(seconds=rounds_without_tire_problem * round_duration_sec)).isoformat()[:-3]}Z"
        
    print(start)
    print(end)
    
    rounds_driven = rounds_driven + rounds_without_tire_problem

    splits.append(dict(Task=f"{i} Stint", Start=start, Finish=end, split_type="stint", rounds_driven=rounds_driven))
    
    # Break before calculating an additional stop
    if rounds_driven > overall_rounds:
        print(rounds_driven)
        break
    
    ############
    ## BOX CALC ##
    ############
    
    # TODO change "rounds_without_tire_problem" if different strats are applied
    fuel_charging_time = ((rounds_without_tire_problem / rounds_fuel) * 100) / __l_per_sec__
    
    # If fueling takes longer than changing tyres, this can be done in parallel so doesnt need to be summed up
    if fuel_charging_time > __tire_change_sec__:
        stop_time = fuel_charging_time
    else:
        stop_time = __tire_change_sec__
    
    start =  f"{datetime.strptime(end, '%Y-%m-%dT%H:%M:%S.%fZ').isoformat()[:-3]}Z"
    end = f"{(datetime.strptime(end, '%Y-%m-%dT%H:%M:%S.%fZ') +  timedelta(seconds=stop_time)).isoformat()[:-3]}Z"
        
    print(start)
    print(end)
    print(fuel_charging_time)
        
    splits.append(dict(Task=f"{i} Stop", Start=start, Finish=end, split_type="box_stop"))
                  
    i+=1

2021-09-27T20:36:47.429Z
2021-09-27T20:44:31.529Z
2021-09-27T20:44:31.529Z
2021-09-27T20:44:44.529Z
13.0
2021-09-27T20:44:44.529Z
2021-09-27T20:52:28.629Z
2021-09-27T20:52:28.629Z
2021-09-27T20:52:41.629Z
13.0
2021-09-27T20:52:41.629Z
2021-09-27T21:00:25.729Z
2021-09-27T21:00:25.729Z
2021-09-27T21:00:38.729Z
13.0
2021-09-27T21:00:38.729Z
2021-09-27T21:08:22.829Z
2021-09-27T21:08:22.829Z
2021-09-27T21:08:35.829Z
13.0
2021-09-27T21:08:35.829Z
2021-09-27T21:16:19.929Z
2021-09-27T21:16:19.929Z
2021-09-27T21:16:32.929Z
13.0
2021-09-27T21:16:32.929Z
2021-09-27T21:24:17.029Z
54.6


In [30]:
df = pd.DataFrame(splits)

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Task", color="split_type", hover_data={'rounds_driven':True})
#fig.update_yaxes(autorange="reversed")


fig.add_vline(x=end, line_width=3, line_dash="dash", line_color="green")




fig.show()